In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [19]:
data_df = pd.read_csv("./data/items_purchased.csv")
data_df

,kiosk_id,product_id,card_hash,date_time,fc_number
0,108,2457,FEaD8IUmESlpzvPRYyZhMyUv/CRtDNIUyprR5w7e/mPkFz...,2017-12-06 22:06:36-08,20.0
1,220,2884,N8JjRgpqKnQ8BDp+XtxBkfM0If/9GBL+rsOalHjHLdIM8B...,2018-08-08 14:18:09-07,NaN
2,182,2937,svL2pR3SC6O+B/Kg+78gJsiKDxvCtSUohDLGjj02tGC0MS...,2017-10-20 18:22:10-07,NaN
3,339,3814,kzaHILrWFehUeo6gjZV1EwvZHNTXGNqTY/Hzv+mL9Bg+vR...,2018-06-13 01:30:06-07,48.0
4,230,4149,4ox+j2exAB/xLlyHteKO/cNoSdLFb3T29mkacbTAf7ECiX...,2017-10-19 16:27:17-07,NaN
5,391,4201,3ExgFAomm7KRC3Ju5zdxHR5sGhaNuvdgXGCis0iD3GbtZG...,2018-08-15 18:22:15-07,48.0
6,232,4111,hOB3nBlofMM/2khWcX9O2CoUZ4YyCFSXB6vbCowurdMCLl...,2018-05-03 07:24:09-07,NaN
7,391,4201,3ExgFAomm7KRC3Ju5zdxHR5sGhaNuvdgXGCis0iD3GbtZG...,2018-08-15 18:22:15-07,48.0
8,391,4201,56e0BQgz6p4MkLYd7Ln7mfigk9t6HxTS0riVabfsuCdL66...,2018-07-20 15:50:31-07,48.0
9,391,4201,4tqsacWv4yYY9d2ntipXbwzxIqDlkf2c2JPqs+Oitm/rVj...,2018-02-01 16:13:54-08,48.0
